In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import sys
sys.path.insert(0, '../../scripts/classifier')
from nets import net
from loader import dwellingsDataset

In [3]:
classifier = net().float()
optimizer = optim.Adam(classifier.parameters(), lr=1e-4)
dataset = dwellingsDataset()
data = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=4)

In [4]:
classifier.train()

net(
  (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (conv2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (conv3): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
  (batchnorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (upsample): Upsample(scale_factor=2.0, mode=bilinear)
  (conv4): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (batchnorm4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (conv5): Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1))
  (batchnorm5): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (logsoftmax): LogSoftmax()
)

In [5]:
optimizer.zero_grad()

In [6]:
for batch in data:
    x = batch[0].float()
    y = batch[1].long()
    break

In [7]:
print(x.shape)
print(y.shape)

torch.Size([100, 4, 128, 128])
torch.Size([100, 128, 128])


In [8]:
output = classifier(x)

/home/lsanmartin/.local/lib/python3.5/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [9]:
output.shape

torch.Size([100, 2, 128, 128])

In [10]:
def loss_nll(predicts, targets):
    '''
    '''

    loss = nn.NLLLoss()
    output = loss(predicts, targets)

    return output

In [11]:
loss = loss_nll(output, y)

In [12]:
loss.item()

0.7430726885795593

In [13]:
y.size()

torch.Size([100, 128, 128])

In [14]:
predictions = torch.argmax(output.data, dim=1).numpy()

In [15]:
num_correct = (predictions == y.numpy()).sum()

In [16]:
num_samples = y.size(0) * y.size(1) * y.size(2)

In [17]:
num_correct / num_samples

0.425369873046875